In [1]:
import os

DEFAULT_CAMERA = os.environ.get('JETBOT_DEFAULT_CAMERA', 'opencv_gst_camera')

In [2]:
print(DEFAULT_CAMERA)

opencv_gst_camera


In [13]:
!systemctl restart nvargus-daemon

System has not been booted with systemd as init system (PID 1). Can't operate.


In [2]:
import cv2
# gstreamer_pipeline returns a GStreamer pipeline for capturing from the CSI camera
# Defaults to 1280x720 @ 60fps
# Flip the image by setting the flip_method (most common values: 0 and 2)
# display_width and display_height determine the size of the window on the screen
def gstreamer_pipeline(
    capture_width=1280,
    capture_height=720,
    display_width=1280,
    display_height=720,
    framerate=60,
    flip_method=0,
):
    return (
        "nvarguscamerasrc ! "
        "video/x-raw(memory:NVMM), "
        "width=(int)%d, height=(int)%d, "
        "format=(string)NV12, framerate=(fraction)%d/1 ! "
        "nvvidconv flip-method=%d ! "
        "video/x-raw, width=(int)%d, height=(int)%d, format=(string)BGRx ! "
        "videoconvert ! "
        "video/x-raw, format=(string)BGR ! appsink"
        % (
            capture_width,
            capture_height,
            framerate,
            flip_method,
            display_width,
            display_height,
        )
    )

def show_camera():
    # To flip the image, modify the flip_method parameter (0 and 2 are the most common)
    print(gstreamer_pipeline(flip_method=0))
    cap = cv2.VideoCapture(gstreamer_pipeline(flip_method=0), cv2.CAP_GSTREAMER)
    if cap.isOpened():
        window_handle = cv2.namedWindow("CSI Camera", cv2.WINDOW_AUTOSIZE)
        # Window
        while cv2.getWindowProperty("CSI Camera", 0) >= 0:
            ret_val, img = cap.read()
            cv2.imshow("CSI Camera", img)
            # This also acts as
            print('Streaming')
            keyCode = cv2.waitKey(30) & 0xFF
            # Stop the program on the ESC key
            if keyCode == 27:
                break
        cap.release()
        cv2.destroyAllWindows()
    else:
        print("Unable to open camera")

In [3]:
print(gstreamer_pipeline(flip_method=0))

nvarguscamerasrc ! video/x-raw(memory:NVMM), width=(int)1280, height=(int)720, format=(string)NV12, framerate=(fraction)60/1 ! nvvidconv flip-method=0 ! video/x-raw, width=(int)1280, height=(int)720, format=(string)BGRx ! videoconvert ! video/x-raw, format=(string)BGR ! appsink


In [4]:
cap = cv2.VideoCapture(gstreamer_pipeline(flip_method=0), cv2.CAP_GSTREAMER)

In [5]:
print(cap.isOpened())

True


In [ ]:
window_handle = cv2.namedWindow("CSI Camera", cv2.WINDOW_AUTOSIZE)

In [6]:
ret_val, img = cap.read()

In [7]:
print(ret_val)

False


In [7]:
print(img)

None


In [6]:
cv2.imshow("CSI Camera", img)

error: OpenCV(4.1.1) /home/nvidia/host/build_opencv/nv_opencv/modules/highgui/src/window.cpp:352: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'


In [1]:
import traitlets
import os
import traitlets
import atexit
import cv2
import threading
import numpy as np
    
class CameraBase(traitlets.HasTraits):
    
    value = traitlets.Any()
    
    @staticmethod
    def instance(*args, **kwargs):
        raise NotImplementedError
    
    def widget(self):
        if hasattr(self, '_widget'):
            return self._widget   # cache widget, so we don't duplicate links
        from ipywidgets import Image
        from jetbot.image import bgr8_to_jpeg
        image = Image()
        traitlets.dlink((self, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
        self._widget = image
        return image

class OpenCvGstCamera(CameraBase):
    
    value = traitlets.Any()
    
    # config
    width = traitlets.Integer(default_value=1280).tag(config=True)
    height = traitlets.Integer(default_value=720).tag(config=True)
    fps = traitlets.Integer(default_value=60).tag(config=True)
    capture_width = traitlets.Integer(default_value=1280).tag(config=True)
    capture_height = traitlets.Integer(default_value=720).tag(config=True)
    flip_method = traitlets.Integer(default_value=0).tag(config=True)

    def __init__(self, *args, **kwargs):
        self.value = np.empty((self.height, self.width, 3), dtype=np.uint8)
        super().__init__(self, *args, **kwargs)

        try:
            self.cap = cv2.VideoCapture(self._gst_str(), cv2.CAP_GSTREAMER)

            re, image = self.cap.read()

            if not re:
                raise RuntimeError('Could not read image from camera.')

            self.value = image
            self.start()
        except:
            self.stop()
            raise RuntimeError(
                'Could not initialize camera.  Please see error trace.')

        atexit.register(self.stop)

    def _capture_frames(self):
        while True:
            re, image = self.cap.read()
            if re:
                self.value = image
            else:
                break
                
#     def _gst_str(self):
#         return 'nvarguscamerasrc sensor-mode=3 ! video/x-raw(memory:NVMM), width=%d, height=%d, format=(string)NV12, framerate=(fraction)%d/1 ! nvvidconv ! video/x-raw, width=(int)%d, height=(int)%d, format=(string)BGRx ! videoconvert ! appsink' % (
#                 self.capture_width, self.capture_height, self.fps, self.width, self.height)
    def _gst_str(self):
        return (
            "nvarguscamerasrc ! "
            "video/x-raw(memory:NVMM), "
            "width=(int)%d, height=(int)%d, "
            "format=(string)NV12, framerate=(fraction)%d/1 ! "
            "nvvidconv flip-method=%d ! "
            "video/x-raw, width=(int)%d, height=(int)%d, format=(string)BGRx ! "
            "videoconvert ! "
            "video/x-raw, format=(string)BGR ! appsink"
            % (
                self.capture_width,
                self.capture_height,
                self.fps,
                self.flip_method,
                self.width,
                self.height,
            )
        )
    
    def start(self):
        if not self.cap.isOpened():
            self.cap.open(self._gst_str(), cv2.CAP_GSTREAMER)
        if not hasattr(self, 'thread') or not self.thread.isAlive():
            self.thread = threading.Thread(target=self._capture_frames)
            self.thread.start()

    def stop(self):
        if hasattr(self, 'cap'):
            self.cap.release()
        if hasattr(self, 'thread'):
            self.thread.join()
            
    def restart(self):
        self.stop()
        self.start()
        
    @staticmethod
    def instance(*args, **kwargs):
        return OpenCvGstCamera(*args, **kwargs)

In [12]:
!sudo systemctl restart nvargus-daemon

/bin/bash: sudo: command not found


In [2]:
camera = OpenCvGstCamera(width=224, height=224)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: DeprecationWarning: Passing unrecognized arguments to super(OpenCvGstCamera).__init__(<__main__.OpenCvGstCamera object at 0x7f744c3ef0>).
object.__init__() takes no parameters
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.


RuntimeError: Could not initialize camera.  Please see error trace.

In [ ]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)

image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

In [5]:
from jetbot.camera.zmq_camera import ZmqCamera
Camera = ZmqCamera

In [4]:
!systemctl restart nvargus-daemon

System has not been booted with systemd as init system (PID 1). Can't operate.


In [3]:
from jetbot import Camera
camera = Camera.instance()

In [4]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import bgr8_to_jpeg

In [8]:
import traitlets
import ipywidgets.widgets as widgets

image = widgets.Image(format='jpeg', width=300, height=300)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

NameError: name 'camera' is not defined

In [ ]:
camera.stop()